In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder,FunctionTransformer

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, cross_val_predict, KFold ,  StratifiedKFold

from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, log_loss, confusion_matrix
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier


In [48]:
def cls_feature(X,max_cat = 10):
        #Agrupando features numéricas
    numeric_features  = [cls for cls in X.columns if X[cls].dtype == 'float64' or X[cls].dtype =='int64']
    # print('>>numeric_features:', numeric_features ,
          # len(numeric_features))


    #Agrupando features categoricas
    categorical_features = [cls for cls in X.columns if X[cls].dtypes == 'object' and X[cls].nunique()<=max_cat]
    # print('\n>>cat_features:', cat_features ,
          # len(cat_features))
    return numeric_features , categorical_features

In [49]:
def print_metrics(y_true, y_pred, y_pred_proba):
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_test, y_pred)
    logloss = log_loss(y_true, y_pred_proba)
    roc_auc = roc_auc_score(y_true, y_pred)

    print("Accuracy: {:.2f}".format(accuracy))
    print("Precision: {:.2f}".format(precision))
    print("Recall: {:.2f}".format(recall))
    print("F1-Score: {:.2f}".format(f1))
    print("Log Loss: {:.2f}".format(logloss))
    print("ROC AUC Score: {:.2f}".format(roc_auc))

In [50]:
def custom_features(df):
    df_out = df.copy()
    df_out['cat_empresa'] = df_out['ano_de_estabelecimento'].apply(lambda x: 'antiga' if x <= 1945 else
                                                                    'intermediaria' if 1945 < x < 1996 else
                                                                    'nova' if x >= 1996 else 'NaN')
    percentil_25 = np.percentile(df['salario_prevalecente'], 25)
    percentil_75 = np.percentile(df['salario_prevalecente'], 75)
    median = np.median(df['salario_prevalecente'])
    df_out['cat_salario'] = df_out['salario_prevalecente'].apply(lambda x: 'baixo' if x < percentil_25 else
                                                                'medio_baixo' if percentil_25 <= x < median else
                                                                'medio_alto' if median <= x < percentil_75 else
                                                                'alto' if x >= percentil_75 else 'NaN')
    
    # df_out['cat_salario'] = df_out['salario_prevalecente'].apply(lambda x: 'baixo' if x < np.percentile(df['salario_prevalecente'], 25) else
    #                                                                'medio_baixo' if np.percentile(df['salario_prevalecente'], 25) <= x < np.median(df['salario_prevalecente']) else
    #                                                                'medio_alto' if np.median(df['salario_prevalecente']) <= x < np.percentile(df['salario_prevalecente'], 75) else
    #                                                                'alto' if x >= np.percentile(df['salario_prevalecente'], 75) else 'NaN')
    df_out['cat_experiencia'] = 'Null'
    df_out.loc[(df_out['tem_experiencia_de_trabalho'] == 'S') & (df_out['requer_treinamento_de_trabalho'] == 'S'), 'cat_experiencia'] = '1'
    df_out.loc[(df_out['tem_experiencia_de_trabalho'] == 'S') & (df_out['requer_treinamento_de_trabalho'] == 'N'), 'cat_experiencia'] = '2'
    df_out.loc[(df_out['tem_experiencia_de_trabalho'] == 'N') & (df_out['requer_treinamento_de_trabalho'] == 'S'), 'cat_experiencia'] = '3'
    df_out.loc[(df_out['tem_experiencia_de_trabalho'] == 'N') & (df_out['requer_treinamento_de_trabalho'] == 'N'), 'cat_experiencia'] = '4'
    
    
    # Alterando a coluna Salario prevcalencente e unidade de salaio
    #df_out['unidade_de_salario'] = ' '
    
    df_out.loc[df_out['unidade_de_salario'] == 'Hora', 'salario_prevalecente'] *= 2080
    df_out.loc[df_out['unidade_de_salario'] == 'Semana', 'salario_prevalecente'] *= 52
    df_out.loc[df_out['unidade_de_salario'] == 'Mes', 'salario_prevalecente'] *= 12
    df_out.loc[df_out['unidade_de_salario'] == 'Ano', 'salario_prevalecente'] *= 1
    df_out['salario_anual'] = df_out['salario_prevalecente']
    
    
    # posição em tempo integral
    
    df_out['posicao_em_tempo_integral'] = df_out['posicao_em_tempo_integral'].map({'S': 1, 'N': 0})
    df_out['efeito_salario_emprego'] = df_out['salario_anual'] * df_out['posicao_em_tempo_integral']
    
    
    # qualificaçã do trabalho
    
    #df_out['qualificado_trabalho'] = df_out['educacao_do_empregado'] + df_out['tem_experiencia_de_trabalho']
    
    # relação salario/empregado
    df_out['salario_em_relacao_ao_num_de_empregados'] = df_out['salario_prevalecente'] / df_out['num_de_empregados']


    return df_out


## Data

In [51]:
data_train = pd.read_csv('data/train.csv'); data_train.head()

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral,status_do_caso
0,EZYV10567,Europa,Ensino Médio,N,S,2087,1855,Sul,69711.24,Ano,S,Negado
1,EZYV5505,Ásia,Mestrado,S,N,5991,2003,Meio-Oeste,52931.38,Ano,S,Aprovado
2,EZYV5207,Ásia,Ensino Médio,N,N,1426,2000,Ilha,110830.21,Ano,S,Negado
3,EZYV7545,Ásia,Ensino Médio,N,N,3846,1992,Meio-Oeste,91884.68,Semana,S,Negado
4,EZYV16071,Ásia,Ensino Superior,S,N,3957,1949,Nordeste,138155.24,Ano,S,Aprovado


In [52]:
data_train.columns

Index(['id_do_caso', 'continente', 'educacao_do_empregado',
       'tem_experiencia_de_trabalho', 'requer_treinamento_de_trabalho',
       'num_de_empregados', 'ano_de_estabelecimento', 'regiao_de_emprego',
       'salario_prevalecente', 'unidade_de_salario',
       'posicao_em_tempo_integral', 'status_do_caso'],
      dtype='object')

In [53]:
df = data_train.copy()

In [54]:
df.drop(columns=['id_do_caso'],inplace = True)

In [55]:
df.head()

,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral,status_do_caso
0,Europa,Ensino Médio,N,S,2087,1855,Sul,69711.24,Ano,S,Negado
1,Ásia,Mestrado,S,N,5991,2003,Meio-Oeste,52931.38,Ano,S,Aprovado
2,Ásia,Ensino Médio,N,N,1426,2000,Ilha,110830.21,Ano,S,Negado
3,Ásia,Ensino Médio,N,N,3846,1992,Meio-Oeste,91884.68,Semana,S,Negado
4,Ásia,Ensino Superior,S,N,3957,1949,Nordeste,138155.24,Ano,S,Aprovado


## Start

In [56]:
df['status_do_caso'].value_counts()

Aprovado    11937
Negado       5899
Name: status_do_caso, dtype: int64

In [57]:
df['status_do_caso'] = df['status_do_caso'].apply(lambda x: 1 if x == 'Aprovado' else 0)

In [58]:
df['status_do_caso'].value_counts()

1    11937
0     5899
Name: status_do_caso, dtype: int64

In [59]:
df.dtypes

continente                         object
educacao_do_empregado              object
tem_experiencia_de_trabalho        object
requer_treinamento_de_trabalho     object
num_de_empregados                   int64
ano_de_estabelecimento              int64
regiao_de_emprego                  object
salario_prevalecente              float64
unidade_de_salario                 object
posicao_em_tempo_integral          object
status_do_caso                      int64
dtype: object

In [60]:
X = df.drop(columns=['status_do_caso'])
y = df['status_do_caso']

In [61]:
df['ano_de_estabelecimento'] = np.log1p(df['ano_de_estabelecimento'])

## Prepro

In [62]:
custom_features(df)

,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral,status_do_caso,cat_empresa,cat_salario,cat_experiencia,salario_anual,efeito_salario_emprego,salario_em_relacao_ao_num_de_empregados
0,Europa,Ensino Médio,N,S,2087,7.526179,Sul,69711.24,Ano,1,0,antiga,medio_baixo,3,69711.24,69711.24,33.402607
1,Ásia,Mestrado,S,N,5991,7.602900,Meio-Oeste,52931.38,Ano,1,1,antiga,medio_baixo,2,52931.38,52931.38,8.835149
2,Ásia,Ensino Médio,N,N,1426,7.601402,Ilha,110830.21,Ano,1,0,antiga,alto,4,110830.21,110830.21,77.721045
3,Ásia,Ensino Médio,N,N,3846,7.597396,Meio-Oeste,4778003.36,Semana,1,0,antiga,medio_alto,4,4778003.36,4778003.36,1242.330567
4,Ásia,Ensino Superior,S,N,3957,7.575585,Nordeste,138155.24,Ano,1,1,antiga,alto,2,138155.24,138155.24,34.914137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17831,Ásia,Mestrado,S,N,95347,7.598900,Oeste,69692.24,Ano,1,1,antiga,medio_baixo,2,69692.24,69692.24,0.730933
17832,Europa,Ensino Médio,N,N,1483,7.542744,Sul,136237.62,Ano,1,0,antiga,alto,4,136237.62,136237.62,91.866231
17833,América do Norte,Mestrado,N,N,1504,7.606387,Nordeste,118187.30,Ano,1,0,antiga,alto,4,118187.30,118187.30,78.581981
17834,América do Norte,Doutorado,S,N,251967,7.606387,Meio-Oeste,93133.40,Ano,1,1,antiga,medio_alto,2,93133.40,93133.40,0.369625


In [63]:
custom_features(df).shape

(17836, 17)

In [64]:
feature_engineering_transformer = FunctionTransformer(custom_features)

In [65]:
numeric_features , categorical_features = cls_feature(custom_features(X))

In [66]:
numeric_features

['num_de_empregados',
 'ano_de_estabelecimento',
 'salario_prevalecente',
 'posicao_em_tempo_integral',
 'salario_anual',
 'efeito_salario_emprego',
 'salario_em_relacao_ao_num_de_empregados']

In [67]:
categorical_features

['continente',
 'educacao_do_empregado',
 'tem_experiencia_de_trabalho',
 'requer_treinamento_de_trabalho',
 'regiao_de_emprego',
 'unidade_de_salario',
 'cat_empresa',
 'cat_salario',
 'cat_experiencia']

In [68]:
categorical_transformer = Pipeline([
    ('imputer',SimpleImputer(strategy='constant',fill_value='missing')),
    ('enconder',OneHotEncoder(handle_unknown='ignore',sparse=False)),
    
])
numerical_transformer = Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


preprocessor = ColumnTransformer([
        ("num", numerical_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
]
)

pipeline_fe = Pipeline(steps=[
    ('fe', feature_engineering_transformer),
    ('preprocessor', preprocessor)])

In [69]:
X_preprocessed_fe = pipeline_fe.fit_transform(X)

In [70]:
X_preprocessed_fe.shape

(17836, 41)

In [71]:
# # Dividir os dados em conjuntos de treinamento e teste
# X_train, X_test, y_train, y_test = train_test_split(X_preprocessed_fe, y, test_size=0.2, random_state=42)

# # Definir os modelos
# models = {
#     'RandomForest': RandomForestClassifier(class_weight='balanced', random_state=42),
#     'XGBoost': XGBClassifier(random_state=42,class_weight='balanced'),
#     'LGBM': LGBMClassifier(random_state=42,class_weight = 'balanced')
# }

# # Definir as grades de hiperparâmetros para cada modelo
# param_grids = {
#     'RandomForest': {
#         'n_estimators': [100, 200, 500,1000],
#         'max_depth': [None, 10, 30],
#         'min_samples_split': [2, 5, 10],
#         'criterion': ['gini', 'entropy']
#     },
#     'XGBoost': {
#         'n_estimators': [100, 200, 500,1000],
#         'learning_rate': [0.001, 0.1, 0.3],
#         'max_depth': [3, 6, 10]
        
#     },
#     'LGBM':{
#         'learning_rate': [0.001, 0.1, 0.3],
#         'num_leaves':[2,16,64,128],
#         'min_child_samples':[1,40,70,100]
        
        
#     }
# }

# # Validação cruzada com 3 folds
# cv = KFold(n_splits=3, shuffle=True, random_state=42)

# # Treinar e ajustar os modelos
# grids = {} # armazenará os modelos
# for model_name, model in models.items():
#     grids[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring='f1', n_jobs=-1, verbose=1)
#     grids[model_name].fit(X_train, y_train)
#     best_params = grids[model_name].best_params_
#     best_score = grids[model_name].best_score_
    
#     # Prever os rótulos do conjunto de teste
#     y_pred = grids[model_name].predict(X_test)
    
#     # Calcular o F1 score para o conjunto de teste
#     f1 = f1_score(y_test, y_pred)
    
#     # Imprimir os resultados
#     print(f'Best parameters for {model_name}: {best_params}')
#     print(f'Best F1 score for {model_name}: {best_score}')
#     print(f'F1 score on test set for {model_name}: {f1}\n')

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed_fe, y, test_size=0.2, random_state=42)

# Definir os modelos
models = {
    'RandomForest': RandomForestClassifier(class_weight='balanced', random_state=42),
    'XGBoost': XGBClassifier(random_state=42)
}

# Definir as grades de hiperparâmetros para cada modelo
param_distributions = {
    'RandomForest': {
        'n_estimators': [100, 200, 500, 1000],
        'max_depth': [None, 10, 30],
        'min_samples_split': [2, 5, 10],
        'criterion': ['gini', 'entropy']
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500, 1000],
        'learning_rate': [1e-3, 1e-1,1],
        'max_depth': [1,3, 6, 10],
        'subsample':[0.5,1],
        'min_child_weight' :[1,10],
        'colsample_bytree':[0.5,1]
        
    }
}

# Validação cruzada com 3 folds
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Treinar e ajustar os modelos
searches = {}
for model_name, model in models.items():
    searches[model_name] = RandomizedSearchCV(estimator=model, param_distributions=param_distributions[model_name], scoring='f1', n_iter=10, cv=cv, random_state=42, n_jobs=-1, verbose=2)
    searches[model_name].fit(X_preprocessed_fe, y)
    best_params = searches[model_name].best_params_
    best_score = searches[model_name].best_score_
    
    
    y_pred = searches[model_name].predict(X_test)

    f1 = f1_score(y_test, y_pred)

    # Imprimir os resultados
    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best F1 score for {model_name}: {best_score}\n')
    print(f'F1 score on test set for {model_name}: {f1}\n')


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters for RandomForest: {'n_estimators': 100, 'min_samples_split': 2, 'max_depth': None, 'criterion': 'gini'}
Best F1 score for RandomForest: 0.8068184348020293

F1 score on test set for RandomForest: 1.0

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters for XGBoost: {'subsample': 0.5, 'n_estimators': 200, 'min_child_weight': 10, 'max_depth': 10, 'learning_rate': 0.001, 'colsample_bytree': 1}
Best F1 score for XGBoost: 0.8238136111962336

F1 score on test set for XGBoost: 0.8344111307074269



In [76]:
best_params

{'subsample': 0.5,
 'n_estimators': 200,
 'min_child_weight': 10,
 'max_depth': 10,
 'learning_rate': 0.001,
 'colsample_bytree': 1}

## melhores modelos

### Random Forest
- Fitting 3 folds for each of 10 candidates, totalling 30 fits
- Best parameters for RandomForest: {'n_estimators': 500, 'min_samples_split': 2, 'max_depth': 30, 'criterion': 'entropy'}
- Best F1 score for RandomForest: 0.8075161410111065
- F1 score on test set for RandomForest: 0.8235983590545028

### XGBoost
- Fitting 3 folds for each of 10 candidates, totalling 30 fits
- Best parameters for XGBoost: {'subsample': 1, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.001, 'colsample_bytree': 0.5}
- Best F1 score for XGBoost: 0.8250651531105628
- F1 score on test set for XGBoost: 0.8235983590545028

### Random Forest
- Fitting 3 folds for each of 54 candidates, totalling 162 fits
- Best parameters for RandomForest: {'criterion': 'gini', 'max_depth': 30, 'min_samples_split': 2, 'n_estimators': 500}
- Best F1 score for RandomForest: 0.803983160450065
- F1 score on test set for RandomForest: 0.8011928429423459

### Xgboost
- Fitting 3 folds for each of 27 candidates, totalling 81 fits
- Best parameters for XGBoost: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
- Best F1 score for XGBoost: 0.8237540956961373
- F1 score on test set for XGBoost: 0.8294094944037051

## Modelos

In [77]:
best_model = XGBClassifier(random_state=42, **best_params)
best_model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.001, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=10, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [78]:
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)
# # Calcular a métrica de avaliação (por exemplo, F1 score) no conjunto de teste
# f1 = f1_score(y_test, y_pred)

# # Imprimir o desempenho do modelo
# print(f'F1 score on test set: {f1}')

In [79]:
print_metrics(y_test,y_pred,y_pred_proba)

Accuracy: 0.75
Precision: 0.79
Recall: 0.87
F1-Score: 0.83
Log Loss: 0.64
ROC AUC Score: 0.69


## Submitions

In [80]:
df_test = pd.read_csv('data/test.csv')

In [81]:
df_test.head()

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral
0,EZYV22339,Ásia,Ensino Superior,S,N,2414,1973,Nordeste,103320.80,Ano,N
1,EZYV9333,Ásia,Mestrado,N,S,2928,1937,Oeste,49786.35,Ano,S
2,EZYV9740,Ásia,Ensino Superior,S,N,1836,1999,Oeste,60855.83,Ano,S
3,EZYV3328,Ásia,Ensino Superior,S,N,122,2004,Sul,82379.06,Semana,S
4,EZYV12321,América do Sul,Ensino Superior,S,S,1720,2003,Nordeste,6719.81,Ano,S


In [82]:
df_test_processed = pipeline_fe.transform(df_test)

In [83]:
df_test['status_do_caso'] = best_model.predict(df_test_processed)

In [84]:
#df_test['status_do_caso'] = grids['XGBoost'].predict(df_test_processed)

In [85]:
submit_resp = df_test[['id_do_caso', 'status_do_caso']]

In [86]:
submit_resp

,id_do_caso,status_do_caso
0,EZYV22339,1
1,EZYV9333,1
2,EZYV9740,1
3,EZYV3328,1
4,EZYV12321,1
...,...,...
7639,EZYV19907,0
7640,EZYV576,0
7641,EZYV1804,1
7642,EZYV13543,1


In [87]:
submit_resp.to_csv('data/subs/submission_6_jun5_12;00.csv', index=False)